In [ ]:
"""
Requirements
- Biopython
- HMMER (Version 3.4 tested; "hmmpress" and "hmmscan" need to be on the system $PATH)
- DIAMOND (Version 2.1.8 tested; "diamond" needs to be on the system $PATH)
"""

In [ ]:
import os
import glob
import shutil
import subprocess
import zipfile
import requests
from Bio import SeqIO
from Bio import SearchIO
from Bio.Blast import NCBIXML

In [ ]:
"""
4.2.1, Step 3
Rename downloaded GenBank files and copy them to a single directory.
Note: The unzipped directory ("ncbi_dataset") needs to be placed in the same directory as this Notebook.
"""
gbk_dir = "gbk_files"
os.makedirs(gbk_dir,exist_ok=True)
gbks = glob.glob("ncbi_dataset/**/genomic.gbff",recursive=True)
for gbk in gbks:
    seq_records = SeqIO.parse(open(gbk),"genbank")
    for record in seq_records:
        fungus = record.annotations['organism'].replace(" ","_")
        break
    shutil.copy(gbk, os.path.join(gbk_dir,f"{fungus}.gbff"))

In [ ]:
"""
4.2.2, Step 1
Extract Pyr4 homologues from BGCs that encode a Pyr4 homomolue and an additional core enzyme as a FASTA file ("proteins.fasta")
"""
fasta = open("proteins.fasta","w")
homologue_list = []

bgc_gbks = glob.glob("results/all_clusters/**/*.gbk",recursive=True)
for gbk in bgc_gbks:
    core_list = []
    seq_record = SeqIO.read(open(gbk),"genbank")
    for feature in seq_record.features:
        if feature.type == "CDS":
            try:
                core = feature.qualifiers["core"][0].split(";")[0]
                if core not in core_list:
                    core_list.append(core)
            except:
                pass
    if len(core_list) > 1:
        for feature in seq_record.features:
            if feature.type == "CDS":
                try:
                    if "Pyr4" in feature.qualifiers["Pfam"][0]:
                        locus_tag = feature.qualifiers["locus_tag"][0]
                        feature_seq = feature.qualifiers["translation"][0].replace("-","")
                        fasta.write(f">{locus_tag}\n")
                        fasta.write(f"{feature_seq}\n")
                        homologue_list.append([locus_tag,feature_seq,gbk])
                except:
                    pass
fasta.close()

In [ ]:
"""
4.2.2, Step 2
Extract Pyr4 homologues from the FunBGCs database
Note: This step requires the FASTA file ("proteins.fasta") created in the previous step.
"""
def runHMMpress(input_file):
    subprocess.run(["hmmpress",input_file],stdout=subprocess.DEVNULL)
def runHMMscan(input_file,output_file,database,Evalue):
    subprocess.run(["hmmscan","--domtblout",output_file,"-E",Evalue,"--domE",
                    Evalue,database,input_file],stdout=subprocess.DEVNULL)
def makeDIAMONDdb(fasta,database):
    subprocess.run(["diamond","makedb","--in",fasta,"--db",database],stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)
def RunDIAMOND(fasta,database,output_file,max_target_seqs,max_hsps):
    max_target_seqs = str(max_target_seqs)
    max_hsps = str(max_hsps)
    subprocess.run(["diamond","blastp","--query",fasta,"--db",database,
                    "--out",output_file,"--outfmt","5","--max-target-seqs",max_target_seqs,
                    "--max-hsps",max_hsps],stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)

temp_dir = "temp"
os.makedirs(temp_dir,exist_ok=True)

#Download Pyr4.hmm and all_proteins.fa and copy them to the temp directory
hmm_url = "https://personal.cityu.edu.hk/ymatsuda/funbgcs/funbgcs/HMM.zip"
# hmm_url = "http://staffweb1.cityu.edu.hk/ymatsuda/funbgcs/funbgcs/HMM.zip" #use this URL if the first one does not work
proteins_url = "https://personal.cityu.edu.hk/ymatsuda/funbgcs/funbgcs/all_proteins.fa"
# proteins_url = "http://staffweb1.cityu.edu.hk/ymatsuda/funbgcs/funbgcs/all_proteins.fa" #use this URL if the first one does not work

response = requests.get(hmm_url)
zip_file_path = "HMM.zip"
with open(zip_file_path, "wb") as file:
    file.write(response.content)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall()

hmm_file = "Pyr4.hmm"
shutil.copy(os.path.join(extracted_folder, hmm_file), f"temp/{hmm_file}")

response2 = requests.get(proteins_url)
with open("temp/all_proteins.fa", "wb") as file:
    file.write(response2.content)

os.remove(zip_file_path)
shutil.rmtree(extracted_folder)

#Perform hmmscan analysis against all proteins in the FunBGCs database
runHMMpress(f"temp/{hmm_file}")

hmmscan_input = "temp/all_proteins.fa"
hmmscan_output = "temp/hmmscan.txt"
hmmscan_database = f"temp/{hmm_file}"
runHMMscan(hmmscan_input,hmmscan_output,hmmscan_database,"1e-5")

hit_list = []
for qresult in SearchIO.parse(hmmscan_output, 'hmmscan3-domtab'):
        for i in range(len(qresult.hits)):
            hit_list.append(qresult.id)

#Create a FASTA file with known Pyr4 homologues
homologues_fasta = open("known_homologues.fasta","w")
all_prot_list = [line for line in open("temp/all_proteins.fa","r")]
for i in range(len(all_prot_list)):
    if all_prot_list[i].startswith(">"):
        prot_name = all_prot_list[i].replace(">","").replace("\n","")
        if prot_name in hit_list:
            homologues_fasta.write(all_prot_list[i])
            homologues_fasta.write(all_prot_list[i+1])
homologues_fasta.close()

#Remove duplicated sequences
known_prot = "known_homologues.fasta"
diamond_database = "temp/known_homologues"
makeDIAMONDdb(known_prot,diamond_database)

extracted_prot = "proteins.fasta"
diamond_output = "temp/diamond_result.xml"
RunDIAMOND(extracted_prot,diamond_database,diamond_output,1,1)

duplicated_proteins = []
blast_records = NCBIXML.parse(open(diamond_output))
for blast_record in blast_records:
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            query = blast_record.query
            blast_hit = alignment.title.replace(" ","")
            identity = 100*hsp.identities/len(hsp.match)
            if identity > 90:
                duplicated_proteins.append(query)

original_fasta_list = [line for line in open("proteins.fasta","r")]
with open("temp/proteins_.fasta","w") as fa:
    for i in range(len(original_fasta_list)):
        if original_fasta_list[i].startswith(">"):
            prot_name = original_fasta_list[i].replace(">","").replace("\n","")            
            if prot_name not in duplicated_proteins:
                fa.write(original_fasta_list[i])
                fa.write(original_fasta_list[i+1])

#Remove duplicated sequences from the FASTA file for known proteins
known_prot = "known_homologues.fasta"
diamond_database = "temp/known_homologues"
makeDIAMONDdb(known_prot,diamond_database)
diamond_output = "temp/diamond_result.xml"
RunDIAMOND(known_prot,diamond_database,diamond_output,20,1)

duplicated_proteins = []
blast_records = NCBIXML.parse(open(diamond_output))
for blast_record in blast_records:
    sublist = []
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            query = blast_record.query
            blast_hit = alignment.title.replace(" ","")
            identity = 100*hsp.identities/len(hsp.match)
            if query != blast_hit and identity > 90:
                NewProtein = True
                for item in duplicated_proteins:
                    if query in item:
                        NewProtein = False
                        if blast_hit not in item:
                            item.append(blast_hit)
                if NewProtein:
                    duplicated_proteins.append([query,blast_hit])

ToBeDeleted = []
for item in duplicated_proteins:
    for i in range(1,len(item)):
        ToBeDeleted.append(item[i])

original_fasta_list = [line for line in open(known_prot,"r")]
with open("temp/known_homologues_.fasta","w") as fa:
    for i in range(len(original_fasta_list)):
        if original_fasta_list[i].startswith(">"):
            prot_name = original_fasta_list[i].replace(">","").replace("\n","")            
            if prot_name not in ToBeDeleted:
                fa.write(original_fasta_list[i])
                fa.write(original_fasta_list[i+1])

#Combine the two FASTA files
extracted_prot = "temp/proteins_.fasta"
known_prot = "temp/known_homologues_.fasta"
combined_prot = "combined.fasta"

with open(combined_prot,"w") as cp:
    with open(extracted_prot,"r") as ep:
        for line in ep:
            cp.write(line)
    with open(known_prot,"r") as kp:
        for line in kp:
            cp.write(line)

shutil.rmtree(temp_dir)